In [4]:
import json
import tabulate

def trace_match(metrics, testname):
    return metrics[testname]['summary']['trace_match']

# engine_kind = ['sm', 'sm-base', 'sm-opt', 'v8', 'v8-liftoff', 'v8-turbofan', 'jsc', 'jsc-int','jsc-bbq','jsc-omg', 'wizeng','wizeng-int','wizeng-jit','wizeng-dyn','wasmtime','wasmer','wasmer-base']
engine_kind = ['sm', 'v8', 'jsc', 'wasmtime','wasmer', 'wizeng-int','wizeng-jit','wizeng-dyn']

with open('metrics.json', 'r') as f:
    metrics = json.load(f)

print('RQ1: Applicability')

print('RQ1-1: Accuracy Experiment')
rq1_results = [['Test Name', 'Trace Match']] + sorted([[testname, 'o' if trace_match(metrics, testname) else ''] for testname in metrics]) + [['Total', sum([1 for testname in metrics if trace_match(metrics, testname)])]]
print(tabulate.tabulate(rq1_results, tablefmt="latex"))

print('RQ1-2: Portability Experiment')
def get_engine_runtime(testname):
    return [metrics[testname]['replay_metrics'][engine]['benchmark'].get('runtime') for engine in engine_kind]
rq22_results = [['Test name', *engine_kind]] + sorted([[testname, *get_engine_runtime(testname)] for testname in metrics if trace_match(metrics, testname) ])
print(tabulate.tabulate(rq22_results, tablefmt="latex"))

RQ1: Applicability
RQ1-1: Accuracy Experiment
\begin{tabular}{ll}
\hline
 Test Name       & Trace Match \\
 boa             & o           \\
 bullet          & o           \\
 commanderkeen   & o           \\
 factorial       & o           \\
 ffmpeg          & o           \\
 fib             & o           \\
 figma-startpage & o           \\
 fractals        &             \\
 funky-kart      & o           \\
 game-of-life    & o           \\
 gotemplate      &             \\
 guiicons        & o           \\
 hnset-bench     &             \\
 hydro           & o           \\
 image-convolute &             \\
 jqkungfu        & o           \\
 jsc             & o           \\
 lichess         &             \\
 livesplit       &             \\
 mandelbrot      & o           \\
 multiplyDouble  & o           \\
 multiplyInt     & o           \\
 ogv             & o           \\
 onnxjs          &             \\
 pacalc          & o           \\
 parquet         & o           \\
 pathfind

In [10]:
import json
import tabulate, scipy.stats

with open('metrics.json', 'r') as f: metrics = json.load(f)
del metrics['funky-kart'] 
print('RQ2: Performance')

print('RQ2-1: Performance Overhead')

def get_runtime_slowdown(testname):
    result = []
    for i in range(5):
        original_runtime = metrics[testname]['record_metrics']['original'][i]['mean'] * metrics[testname]['record_metrics']['original'][i]['samples']
        record_runtime = metrics[testname]['record_metrics']['instrumented'][i]['mean'] * metrics[testname]['record_metrics']['instrumented'][i]['samples']
        result.append(record_runtime/original_runtime)
    return result
def get_cycles_slowdown(testname):
    result = []
    for i in range(5):
        original_cycles = sum(metrics[testname]['record_metrics']['original'][i]['cycles'] if metrics[testname]['record_metrics']['original'][i]['cycles'] != -1 else [-1])
        record_cycles = sum(metrics[testname]['record_metrics']['instrumented'][i]['cycles'] if metrics[testname]['record_metrics']['instrumented'][i]['cycles'] != -1 else [-1])
        result.append(record_cycles/original_cycles)
    return result
runtime_slowdown = [['Test Name', 'Runtime Slowdown[1]', 'Runtime Slowdown[2]', 'Runtime Slowdown[3]','Runtime Slowdown[4]','Runtime Slowdown[5]']] + sorted([[testname, *get_runtime_slowdown(testname)] for testname in metrics if trace_match(metrics, testname)])
gmean_row = ['Geometric Mean']
for i in range(1, len(runtime_slowdown[0])):
    column_values = [row[i] for row in runtime_slowdown[1:]]
    print(column_values)
    column_gmean = scipy.stats.gmean(column_values)
    gmean_row.append(column_gmean)
runtime_slowdown.append(gmean_row)
print(tabulate.tabulate(runtime_slowdown, tablefmt="latex"))

cycles_slowdown = [['Test Name', 'Cycles Slowdown[1]', 'Cycles Slowdown[2]', 'Cycles Slowdown[3]','Cycles Slowdown[4]','Cycles Slowdown[5]']] + sorted([[testname, *get_cycles_slowdown(testname)] for testname in metrics if trace_match(metrics, testname)])
gmean_row = ['Geometric Mean']
for i in range(1, len(cycles_slowdown[0])):
    column_values = [row[i] for row in cycles_slowdown[1:]]
    column_gmean = scipy.stats.gmean(column_values)
    gmean_row.append(column_gmean)
cycles_slowdown.append(gmean_row)
print(tabulate.tabulate(cycles_slowdown, tablefmt="latex"))


print('RQ2-2: Performance Characteristics')
def get_metric(testname, metric):
    return metrics[testname]['replay_metrics']['wizeng-int']['benchmark'].get(metric)
rq22_results = [['Test name', 'replay proportion', 'pregen time', 'load time', 'validate time', 'spc time', 'start time', 'main time']] + [[testname, 'TODO', get_metric(testname, 'pregen:time_us'), get_metric(testname, 'load:time_us'), get_metric(testname, 'validate:time_us'), get_metric(testname, 'spc:time_us'), get_metric(testname, 'start:time_us'), get_metric(testname, 'main:time_us')] for testname in metrics if trace_match(metrics, testname) ]

print(tabulate.tabulate(rq22_results, tablefmt="latex"))

RQ2: Performance
RQ2-1: Performance Overhead


KeyError: 'trace_match'

In [5]:
import json
import tabulate, scipy.stats

with open('metrics.json', 'r') as f: metrics = json.load(f)
print('RQ3: Effectiveness of Trace Reduction')

print('TODO')

RQ3: Effectiveness of Trace Reduction
TODO


In [8]:
import json
import tabulate
with open('metrics.json', 'r') as f:
    metrics = json.load(f)

print('RQ4: Effectiveness of Replay Optimization')

def get_metric(testname, opt, time):
    metric = metrics[testname]['replay_metrics']['wizeng-int'][opt]
    if len(metric) == 0:
        return 0
    else:
        return metric[time]

print('RQ4-1: Load time')
time = 'load:time_us'
rq4_results = [['Test name', 'noopt time', 'split time', 'merge time', 'fullopt time']] + [[testname, get_metric(testname, 'noopt', time), get_metric(testname, 'split', time), get_metric(testname, 'merge', time), get_metric(testname, 'benchmark', time)] for testname in metrics if trace_match(metrics, testname)]
print(tabulate.tabulate(rq4_results, tablefmt="latex"))

print('RQ4-2: Validate time')
time = 'validate:time_us'
rq4_results = [['Test name', 'noopt time', 'split time', 'merge time', 'fullopt time']] + [[testname, get_metric(testname, 'noopt', time), get_metric(testname, 'split', time), get_metric(testname, 'merge', time), get_metric(testname, 'benchmark', time)] for testname in metrics if trace_match(metrics, testname)]
print(tabulate.tabulate(rq4_results, tablefmt="latex"))

print('RQ4-3: Main time')
time = 'main:time_us'
rq4_results = [['Test name', 'noopt time', 'split time', 'merge time', 'fullopt time']] + [[testname, get_metric(testname, 'noopt', time), get_metric(testname, 'split', time), get_metric(testname, 'merge', time), get_metric(testname, 'benchmark', time)] for testname in metrics if trace_match(metrics, testname)]
print(tabulate.tabulate(rq4_results, tablefmt="latex"))

RQ4: Effectiveness of Replay Optimization
RQ4-1: Load time
\begin{tabular}{lllll}
\hline
 Test name       & noopt time & split time & merge time & fullopt time \\
 bullet          & 12240      & 12296      & 11755      & 11719        \\
 jsc             & 95584      & 95274      & 88448      & 88416        \\
 rguistyler      & 9086       & 9090       & 8857       & 8899         \\
 video           & 399        & 398        & 413        & 417          \\
 multiplyDouble  & 2207       & 2293       & 2179       & 2190         \\
 factorial       & 798        & 797        & 792        & 792          \\
 commanderkeen   & 77235      & 77433      & 65214      & 65065        \\
 mandelbrot      & 14804      & 14288      & 1535       & 1537         \\
 sqlpractice     & 0          & 0          & 0          & 0            \\
 rguilayout      & 8830       & 8622       & 8491       & 8606         \\
 riconpacker     & 8577       & 8665       & 8385       & 8444         \\
 figma-startpage & 3960